# 00 - 数据预处理

本notebook用于将原始电池数据集预处理为统一格式。

## 支持的数据集

| 数据集 | 来源 | 电池类型 | 特点 |
|--------|------|----------|------|
| MATR | MIT-Stanford-Toyota | LFP | 多种快充协议 |
| CALCE | 马里兰大学 | LCO | 长期循环数据 |
| HUST | 华中科技大学 | LFP | 多种工况 |
| NASA | NASA PCoE | LCO | 经典基准数据集 |
| XJTU | 西安交通大学 | LFP | 多批次数据 |
| TJU | 天津大学 | NCM | 不同SOC区间 |
| RWTH | 亚琛工业大学 | NCA | 温度实验 |

## 统一数据格式

所有数据预处理后保存为统一的`BatteryData`格式：
- 每个电池一个`.pkl`文件
- 包含完整的充电曲线（电压、电流、时间）
- 已计算SOH

In [ ]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
from src.data import (
    preprocess_dataset,
    preprocess_all,
    load_processed_batteries,
    load_all_processed,
    PREPROCESSORS,
)

# 数据目录配置
RAW_DATA_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')

print(f"原始数据目录: {RAW_DATA_DIR}")
print(f"处理后目录: {PROCESSED_DIR}")
print(f"\n支持的数据集: {list(PREPROCESSORS.keys())}")

## 1. 预处理单个数据集

In [ ]:
# 预处理指定数据集
dataset_name = 'CALCE'  # 可选: MATR, CALCE, HUST, NASA, XJTU, TJU, RWTH

batteries = preprocess_dataset(
    dataset_name=dataset_name,
    raw_dir=str(RAW_DATA_DIR / dataset_name),
    output_dir=str(PROCESSED_DIR / dataset_name),
    verbose=True,
)

print(f"\n预处理完成: {len(batteries)} 个电池")

## 2. 批量预处理所有数据集

In [ ]:
# 预处理所有可用的数据集
all_batteries = preprocess_all(
    raw_base_dir=str(RAW_DATA_DIR),
    output_base_dir=str(PROCESSED_DIR),
    verbose=True,
)

print("\n=== 预处理汇总 ===")
for name, bats in all_batteries.items():
    print(f"  {name}: {len(bats)} 电池")

## 3. 加载已处理的数据

In [ ]:
# 加载单个数据集
batteries = load_processed_batteries(str(PROCESSED_DIR / 'MATR'))
print(f"加载了 {len(batteries)} 个电池")

# 查看第一个电池的信息
if batteries:
    bat = batteries[0]
    print(bat.summary())

In [ ]:
# 加载所有数据集
all_data = load_all_processed(str(PROCESSED_DIR))

print("=== 所有数据集 ===")
total = 0
for name, bats in all_data.items():
    total += len(bats)
    print(f"  {name}: {len(bats)} 电池")
print(f"\n总计: {total} 电池")

## 4. 数据探索

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']  # Windows 系统推荐使用 SimHei
matplotlib.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
# 选择一个电池进行可视化
bat = batteries[0]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. SOH衰减曲线
ax = axes[0, 0]
soh = bat.get_soh_array()
ax.plot(soh, 'b-', linewidth=1)
ax.axhline(y=0.8, color='r', linestyle='--', label='EOL阈值(80%)')
ax.set_xlabel('循环数')
ax.set_ylabel('SOH')
ax.set_title(f'SOH衰减曲线 - {bat.cell_id}')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. 不同循环的充电曲线
ax = axes[0, 1]
cycles_to_plot = [0, len(bat)//4, len(bat)//2, 3*len(bat)//4, len(bat)-1]
for idx in cycles_to_plot:
    if idx < len(bat):
        cycle = bat.cycles[idx]
        ax.plot(cycle.time, cycle.voltage, label=f'Cycle {cycle.cycle_number}')
ax.set_xlabel('时间 (s)')
ax.set_ylabel('电压 (V)')
ax.set_title('不同循环的充电曲线')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. 容量分布
ax = axes[1, 0]
capacities = bat.get_capacity_array()
ax.hist(capacities, bins=30, alpha=0.7, color='steelblue')
ax.set_xlabel('容量 (Ah)')
ax.set_ylabel('频数')
ax.set_title('容量分布')
ax.grid(True, alpha=0.3)

# 4. 充电曲线热力图
ax = axes[1, 1]
from src.data import create_heatmap
heatmap = create_heatmap(bat, window_size=min(100, len(bat)), num_samples=100)
im = ax.imshow(heatmap[:, :, 0], aspect='auto', cmap='viridis')
ax.set_xlabel('采样点')
ax.set_ylabel('循环数')
ax.set_title('电压热力图')
plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## 5. 数据统计

统计各数据集的关键信息。

In [ ]:
# 统计各数据集信息
stats = []
for name, bats in all_data.items():
    total_cycles = sum(len(b) for b in bats)
    eol_count = sum(1 for b in bats if b.eol_cycle is not None)
    avg_cycles = total_cycles / len(bats) if bats else 0
    
    stats.append({
        '数据集': name,
        '电池数': len(bats),
        '总循环数': total_cycles,
        '平均循环数': f'{avg_cycles:.0f}',
        '已达EOL': f'{eol_count}/{len(bats)}',
    })

import pandas as pd
df_stats = pd.DataFrame(stats)
print(df_stats.to_string(index=False))

## 下一步

数据预处理完成后，可以继续：
1. `01_feature_extraction.ipynb` - 特征提取与可视化
2. `02_model_training.ipynb` - 模型训练与SOH/RUL预测